In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn

import librosa
import librosa.display
import IPython.display as ipd
import warnings
warnings.filterwarnings('ignore')

In [2]:
sound_features = pd.read_csv("features_all_labels.csv")
sound_features.drop(['Unnamed: 0'], axis=1, inplace=True)

In [3]:
sound_features

,path,instrument,emotion,f_zero_crossings_sum,f_tempo,f_chroma_stft_mean,f_chroma_stft_var,f_chroma_cqt_mean,f_chroma_cqt_var,f_chroma_cens_mean,...,f_poly_2_var,f_tempogram_mean,f_tempogram_var,f_fourier_tempogram_mean_real,f_fourier_tempogram_mean_imag,f_fourier_tempogram_var,f_harm_mean,f_harm_var,f_perc_mean,f_perc_var
0,/home/ayan-sinha/Desktop/CVRamanLab/Data/medha...,flute,anxiety,10333,117.453835,0.258224,0.098745,0.355288,0.073224,0.245446,...,2.976036,0.153620,0.035368,0.585484,0.009401,529.72450,-1.127914e-05,0.013111,-0.000123,0.000860
1,/home/ayan-sinha/Desktop/CVRamanLab/Data/medha...,flute,anxiety,11497,123.046875,0.237622,0.098343,0.380657,0.073265,0.258766,...,3.200005,0.140677,0.030350,0.610382,0.005564,614.91833,-4.275438e-05,0.013628,-0.000234,0.000619
2,/home/ayan-sinha/Desktop/CVRamanLab/Data/medha...,flute,anxiety,9886,117.453835,0.269323,0.112593,0.355649,0.075457,0.245970,...,3.096585,0.155980,0.036259,0.576286,0.005338,503.57983,1.314697e-05,0.014331,-0.000292,0.000574
3,/home/ayan-sinha/Desktop/CVRamanLab/Data/medha...,flute,anxiety,11491,172.265625,0.231684,0.093056,0.382025,0.064792,0.265052,...,0.333460,0.140639,0.030910,0.763477,0.017326,972.54380,-1.136094e-03,0.000954,-0.000566,0.000128
4,/home/ayan-sinha/Desktop/CVRamanLab/Data/medha...,flute,anxiety,12285,184.570312,0.191707,0.088536,0.418141,0.071533,0.265087,...,0.401022,0.136021,0.027755,0.799386,0.023571,1114.44630,-1.017552e-03,0.001183,-0.000657,0.000174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5609,/home/ayan-sinha/Desktop/CVRamanLab/Data/medha...,sarod,sad,12035,129.199219,0.233833,0.085907,0.287845,0.080724,0.207663,...,0.060125,0.145631,0.034920,0.557559,-0.029431,628.63740,-1.157843e-06,0.000213,-0.000012,0.000006
5610,/home/ayan-sinha/Desktop/CVRamanLab/Data/medha...,sarod,sad,12454,107.666016,0.245914,0.091502,0.290722,0.079875,0.208094,...,0.060977,0.147539,0.033305,0.516971,0.009663,435.61163,-3.330008e-08,0.000240,-0.000015,0.000006
5611,/home/ayan-sinha/Desktop/CVRamanLab/Data/medha...,sarod,sad,11941,234.907670,0.235876,0.094154,0.279254,0.082665,0.201635,...,0.059164,0.151024,0.033891,0.513517,-0.006917,403.71008,-1.003873e-06,0.000251,-0.000016,0.000005
5612,/home/ayan-sinha/Desktop/CVRamanLab/Data/medha...,sarod,sad,23319,89.102909,0.218647,0.087617,0.365741,0.078662,0.244841,...,2.096623,0.093263,0.016462,0.658194,0.013728,960.32310,-1.369764e-05,0.005117,-0.000107,0.001069


In [4]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier, XGBRFClassifier
from xgboost import plot_tree, plot_importance

from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE

In [5]:
def model_assess(model, title = "Default"):
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    #print(confusion_matrix(y_test, preds))
    print('Accuracy', title, ':', round(accuracy_score(y_test, preds), 5), '\n')

In [6]:
data = sound_features.iloc[0:, 1:]
data

,instrument,emotion,f_zero_crossings_sum,f_tempo,f_chroma_stft_mean,f_chroma_stft_var,f_chroma_cqt_mean,f_chroma_cqt_var,f_chroma_cens_mean,f_chroma_cens_var,...,f_poly_2_var,f_tempogram_mean,f_tempogram_var,f_fourier_tempogram_mean_real,f_fourier_tempogram_mean_imag,f_fourier_tempogram_var,f_harm_mean,f_harm_var,f_perc_mean,f_perc_var
0,flute,anxiety,10333,117.453835,0.258224,0.098745,0.355288,0.073224,0.245446,0.023090,...,2.976036,0.153620,0.035368,0.585484,0.009401,529.72450,-1.127914e-05,0.013111,-0.000123,0.000860
1,flute,anxiety,11497,123.046875,0.237622,0.098343,0.380657,0.073265,0.258766,0.016373,...,3.200005,0.140677,0.030350,0.610382,0.005564,614.91833,-4.275438e-05,0.013628,-0.000234,0.000619
2,flute,anxiety,9886,117.453835,0.269323,0.112593,0.355649,0.075457,0.245970,0.022832,...,3.096585,0.155980,0.036259,0.576286,0.005338,503.57983,1.314697e-05,0.014331,-0.000292,0.000574
3,flute,anxiety,11491,172.265625,0.231684,0.093056,0.382025,0.064792,0.265052,0.013081,...,0.333460,0.140639,0.030910,0.763477,0.017326,972.54380,-1.136094e-03,0.000954,-0.000566,0.000128
4,flute,anxiety,12285,184.570312,0.191707,0.088536,0.418141,0.071533,0.265087,0.013062,...,0.401022,0.136021,0.027755,0.799386,0.023571,1114.44630,-1.017552e-03,0.001183,-0.000657,0.000174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5609,sarod,sad,12035,129.199219,0.233833,0.085907,0.287845,0.080724,0.207663,0.040210,...,0.060125,0.145631,0.034920,0.557559,-0.029431,628.63740,-1.157843e-06,0.000213,-0.000012,0.000006
5610,sarod,sad,12454,107.666016,0.245914,0.091502,0.290722,0.079875,0.208094,0.040030,...,0.060977,0.147539,0.033305,0.516971,0.009663,435.61163,-3.330008e-08,0.000240,-0.000015,0.000006
5611,sarod,sad,11941,234.907670,0.235876,0.094154,0.279254,0.082665,0.201635,0.042677,...,0.059164,0.151024,0.033891,0.513517,-0.006917,403.71008,-1.003873e-06,0.000251,-0.000016,0.000005
5612,sarod,sad,23319,89.102909,0.218647,0.087617,0.365741,0.078662,0.244841,0.023386,...,2.096623,0.093263,0.016462,0.658194,0.013728,960.32310,-1.369764e-05,0.005117,-0.000107,0.001069


In [7]:
y = data['instrument'] # genre variable.
X = data.loc[:, np.bitwise_and(data.columns != 'instrument', data.columns != 'emotion')] #select all columns but not the labels

In [8]:
X.columns

Index(['f_zero_crossings_sum', 'f_tempo', 'f_chroma_stft_mean',
       'f_chroma_stft_var', 'f_chroma_cqt_mean', 'f_chroma_cqt_var',
       'f_chroma_cens_mean', 'f_chroma_cens_var', 'f_melspec_mean',
       'f_melspec_var', 'f_mfcc_3_mean', 'f_mfcc_3_var', 'f_mfcc_5_mean',
       'f_mfcc_5_var', 'f_mfcc_8_mean', 'f_mfcc_8_var', 'f_mfcc_10_mean',
       'f_mfcc_10_var', 'f_mfcc_11_mean', 'f_mfcc_11_var', 'f_mfcc_12_mean',
       'f_mfcc_12_var', 'f_mfcc_13_mean', 'f_mfcc_13_var', 'f_mfcc_14_mean',
       'f_mfcc_14_var', 'f_mfcc_15_mean', 'f_mfcc_15_var', 'f_mfcc_16_mean',
       'f_mfcc_16_var', 'f_mfcc_17_mean', 'f_mfcc_17_var', 'f_mfcc_18_mean',
       'f_mfcc_18_var', 'f_mfcc_19_mean', 'f_mfcc_19_var', 'f_mfcc_20_mean',
       'f_mfcc_20_var', 'f_rms_mean', 'f_spec_centr_mean', 'f_spec_centr_var',
       'f_spec_bandw_mean', 'f_spec_bandw_var', 'f_spec_contr_mean',
       'f_spec_contr_var', 'f_spec_flat_mean', 'f_spec_flat_var',
       'f_spec_roll_mean', 'f_spec_roll_var', 'f_ton

In [9]:
#### NORMALIZE X ####

# Normalize so everything is on the same scale. 

cols = X.columns
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(X)

# new data frame with the new scaled data. 
X = pd.DataFrame(np_scaled, columns = cols)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [11]:
# Naive Bayes
nb = GaussianNB()
model_assess(nb, "Naive Bayes")

# Stochastic Gradient Descent
sgd = SGDClassifier(max_iter=5000, random_state=0)
model_assess(sgd, "Stochastic Gradient Descent")

# KNN
knn = KNeighborsClassifier(n_neighbors=19)
model_assess(knn, "KNN")

# Decission trees
tree = DecisionTreeClassifier()
model_assess(tree, "Decission trees")

# Random Forest
rforest = RandomForestClassifier(n_estimators=1000, max_depth=10, random_state=0)
model_assess(rforest, "Random Forest")

# Support Vector Machine
svm = SVC(decision_function_shape="ovo")
model_assess(svm, "Support Vector Machine")

# Logistic Regression
lg = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')
model_assess(lg, "Logistic Regression")

# Neural Nets
nn = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5000, 10), random_state=1)
model_assess(nn, "Neural Nets")

# Cross Gradient Booster (Random Forest)
xgbrf = XGBRFClassifier(objective= 'multi:softmax')
model_assess(xgbrf, "Cross Gradient Booster (Random Forest)")

Accuracy Naive Bayes : 0.62018 

Accuracy Stochastic Gradient Descent : 0.86647 

Accuracy KNN : 0.92582 

Accuracy Decission trees : 0.88724 

Accuracy Random Forest : 0.96142 

Accuracy Support Vector Machine : 0.91395 

Accuracy Logistic Regression : 0.87062 

Accuracy Neural Nets : 0.89614 

[19:56:11] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy Cross Gradient Booster (Random Forest) : 0.93234 



In [12]:
y = data['emotion'] # genre variable.
X = data.loc[:, np.bitwise_and(data.columns != 'instrument', data.columns != 'emotion')] #select all columns but not the labels

In [13]:
#### NORMALIZE X ####

# Normalize so everything is on the same scale. 

cols = X.columns
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(X)

# new data frame with the new scaled data. 
X = pd.DataFrame(np_scaled, columns = cols)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [15]:
# Naive Bayes
nb = GaussianNB()
model_assess(nb, "Naive Bayes")

# Stochastic Gradient Descent
sgd = SGDClassifier(max_iter=5000, random_state=0)
model_assess(sgd, "Stochastic Gradient Descent")

# KNN
knn = KNeighborsClassifier(n_neighbors=19)
model_assess(knn, "KNN")

# Decission trees
tree = DecisionTreeClassifier()
model_assess(tree, "Decission trees")

# Random Forest
rforest = RandomForestClassifier(n_estimators=1000, max_depth=10, random_state=0)
model_assess(rforest, "Random Forest")

# Support Vector Machine
svm = SVC(decision_function_shape="ovo")
model_assess(svm, "Support Vector Machine")

# Logistic Regression
lg = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')
model_assess(lg, "Logistic Regression")

# Neural Nets
nn = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5000, 10), random_state=1)
model_assess(nn, "Neural Nets")

# Cross Gradient Booster (Random Forest)
xgbrf = XGBRFClassifier(objective= 'multi:softmax')
model_assess(xgbrf, "Cross Gradient Booster (Random Forest)")

Accuracy Naive Bayes : 0.41662 

Accuracy Stochastic Gradient Descent : 0.48368 

Accuracy KNN : 0.73412 

Accuracy Decission trees : 0.77567 

Accuracy Random Forest : 0.8457 

Accuracy Support Vector Machine : 0.65935 

Accuracy Logistic Regression : 0.58813 

Accuracy Neural Nets : 0.67893 

[19:57:50] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy Cross Gradient Booster (Random Forest) : 0.80297 

